In [33]:

import re
import numpy as np
from lmfit import minimize, Parameters, fit_report
import matplotlib.pyplot as plt
import pandas as pd
from typing import List

from lib.transformations import (euler_matrix, 
                                 euler_from_matrix, 
                                 scale_from_matrix,
                                 )

def convert_to_homogeneous(x: np.ndarray) -> np.ndarray:
    ''' Convert 3d points in euclidean coordinates (nx3 numpy array) homogenous coordinates (nx4 numpy array)
    '''
    if x.shape[1] != 3: 
        raise ValueError('Wrong dimension of the input array, please provide nx3 numpy array')
    n = x.shape[0]
    x = np.concatenate((x, np.ones((n, 1))), 1, )
     
    return x


def read_data_to_df(
    file_path: str,
    delimiter: str = ',',
    header: int = 0,
    col_names: List[str] = None,
    index_col=None,
) -> pd.DataFrame:
    '''
    '''
    df = pd.read_csv(file_path,
                     sep=delimiter,
                     header=header,
                     names=col_names,
                     index_col=index_col)
    return df


def residuals(params: Parameters,
              x0: np.ndarray, # Points in the starting reference system
              x1: np.ndarray, # Points in final reference system 
              uncertainty: np.ndarray = None # A-priori observation uncertainty
            ) -> np.ndarray:
    ''' 3D rototranslation with scale factor
        X1_ = T_ + m * R * X0_
    '''    
        
    # Get parameters  
    parvals = params.valuesdict()
    rx = parvals['rx']
    ry = parvals['ry']
    rz = parvals['rz']
    tx = parvals['tx']
    ty = parvals['ty']
    tz = parvals['tz']
    m = parvals['m']
    
    # Convert points to homogeneos coordinates and traspose np array to obtain a 4xn matrix
    x0 = convert_to_homogeneous(x0).T

    # Build 4x4 transformation matrix (T) in homogeneous coordinates
    T = np.identity(4)
    R = euler_matrix(rx, ry, rz)
    T[0:3, 0:3] = (m * np.identity(3)) @ R[:3,:3]
    T[0:3, 3:4] = np.array([tx, ty, tz]).reshape(3, 1)
    
    # Apply transformation to x0 points
    x1_ = T @ x0
    x1_ = x1_[:3,:].T 

    # Compute residuals as differences between observed and estimated values, scaled by the a-priori observation uncertainties
    res = (x1 - x1_)
    
    use_weigths = True
    
    if use_weigths:

        sigma0 = 1  # 0.005 # [m]
        uncertainty = np.concatenate((
            np.tile(1, (4, 3)),
            np.tile(10, (2, 3))
        ),
            axis=0,
        )
        
        weighted = res / (sigma0 * uncertainty)
        # weighted = np.sqrt(res ** 2 / uncertainty ** 2)
        # print(weighted)
        
        return weighted.flatten()
    
    else:
        return res.flatten()
    

    ## Not vectorialized approach
    # num_pts = x0.shape[0]
    # R = euler_matrix(rx, ry, rz)
    # res = []
    # for i in range(num_pts):
    #     x_ = tx + m * np.matmul(R[0, :3].reshape(1, 3), x0[i,:].reshape(3, 1))
    #     y_ = ty + m * np.matmul(R[1, :3].reshape(1, 3), x0[i,:].reshape(3, 1))
    #     z_ = tz + m * np.matmul(R[2, :3].reshape(1, 3), x0[i,:].reshape(3, 1))
    #     rx = x1[i, 0] - x_[0][0]
    #     ry = x1[i, 1] - y_[0][0]
    #     rz = x1[i, 2] - z_[0][0]
    #     res_i = np.array([rx, ry, rz])     
    #     res.append(res_i)
    # return np.array(res).flatten()

# minner = Minimizer(residuals, params)
# result = minner.minimize(method='leastsq')


pt_loc = read_data_to_df('../data/lingua_loc.txt',
                         header=None,
                         col_names=['label', 'x', 'y', 'z'],
                         index_col=0
                         )
# pt_loc.to_numpy()
pt_utm = read_data_to_df('../data/lingua_utm.txt',
                         header=None,
                         col_names=['label', 'x', 'y', 'z'],
                         index_col=0
                         )
# pt_utm.to_numpy()

t_ini = np.array([16000., 90000., 1700.], dtype='float64')
rot_ini = np.array([0., 0., 0.], 'float64')
m_ini = float(1.0)

# Define Parameters to be optimized
params = Parameters()
params.add('rx', value=rot_ini[0], vary=False)
params.add('ry', value=rot_ini[1], vary=False)
params.add('rz', value=rot_ini[2], vary=True)
params.add('tx', value=t_ini[0], vary=True)
params.add('ty', value=t_ini[1], vary=True)
params.add('tz', value=t_ini[2], vary=True)
params.add('m',  value=m_ini, vary=True)

# Run Optimization!
result = minimize(residuals, params, args=(
    pt_loc.to_numpy(), pt_utm.to_numpy()),
    scale_covar=True
)

# Print result
print('-------------------------------')
print('Optimization report')
print(fit_report(result))

print('-------------------------------')
print(result.params.pretty_print())
# print('-------------------------------')
# print('Parameter    Value       Stderr')
# for name, param in out.params.items():
#     print(f'{name:7s} {param.value:11.5f} {param.stderr:11.5f}')

print('-------------------------------')
# print(f'Residuals:\n {result.residual.reshape(-1,3)}')
print('Residuals')
print('     X       Y      Z')
for res in result.residual.reshape(-1, 3):
    print(f'{res[0]:8.3f} {res[1]:8.3f} {res[2]:8.3f}')


# print(f'Covariance matrix: \n {result.covar}')





-------------------------------
Optimization report
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 31
    # data points      = 18
    # variables        = 5
    chi-square         = 2.7709e-04
    reduced chi-square = 2.1315e-05
    Akaike info crit   = -189.467678
    Bayesian info crit = -185.015819
[[Variables]]
    rx:  0 (fixed)
    ry:  0 (fixed)
    rz:  0.78558784 +/- 6.0581e-05 (0.01%) (init = 0)
    tx:  16614.8201 +/- 0.01120996 (0.00%) (init = 16000)
    ty:  90932.7192 +/- 0.01055494 (0.00%) (init = 90000)
    tz:  1767.50956 +/- 0.00515225 (0.00%) (init = 1700)
    m:   0.99947914 +/- 5.6848e-05 (0.01%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(rz, tx) = 0.979
    C(ty, m)  = -0.976
    C(tz, m)  = -0.895
    C(ty, tz) = 0.873
-------------------------------
Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
m     0.9995     -inf      inf 5.685e-05     True     None     None
rx         0     -

/home/francesco/anaconda3/envs/rotra3d/lib/python3.8/site-packages/lmfit/printfuncs.py:175: RuntimeWarning: invalid value encountered in double_scalars
  spercent = f'({abs(par.stderr/par.value):.2%})'


In [32]:
def comp_res(params: Parameters,
              x0: np.ndarray,  # Points in the starting reference system
              x1: np.ndarray,  # Points in final reference system
              uncertainty: np.ndarray = None  # A-priori observation uncertainty
              ) -> np.ndarray:
    ''' 3D rototranslation with scale factor
        X1_ = T_ + m * R * X0_
    '''

    # Get parameters
    parvals = params.valuesdict()
    rx = parvals['rx']
    ry = parvals['ry']
    rz = parvals['rz']
    tx = parvals['tx']
    ty = parvals['ty']
    tz = parvals['tz']
    m = parvals['m']

    # Convert points to homogeneos coordinates and traspose np array to obtain a 4xn matrix
    x0 = convert_to_homogeneous(x0).T

    # Build 4x4 transformation matrix (T) in homogeneous coordinates
    T = np.identity(4)
    R = euler_matrix(rx, ry, rz)
    T[0:3, 0:3] = (m * np.identity(3)) @ R[:3, :3]
    T[0:3, 3:4] = np.array([tx, ty, tz]).reshape(3, 1)

    # Apply transformation to x0 points
    x1_ = T @ x0
    x1_ = x1_[:3, :].T

    # Compute residuals as differences between observed and estimated values, scaled by the a-priori observation uncertainties

    uncertainty = np.concatenate((
        np.tile(2e-3, (4, 3)),
        np.tile(3e-2, (2, 3))
    ),
        axis=0,
    )
    
    
    res = (x1 - x1_)
    
    return res
    # print(res)
    # res = res / (uncertainty**2)

    # # res = (x1 - x1_) / (uncertainty**2)



    # weighted = np.sqrt(res ** 2 / uncertainty ** 2)
    
    # print(res)
    # print(weighted)

    # return weighted.flatten()


comp_res(params,
    pt_loc.to_numpy(), pt_utm.to_numpy()
    )



array([[ 501.0676    , 1007.1224    ,   67.46804   ],
       [ 461.9887    , 1008.077     ,   67.46777   ],
       [ 514.7965    ,  974.0701    ,   67.4669    ],
       [ 473.1496    ,  964.5524    ,   67.4668    ],
       [ 464.4492    ,  911.4845    ,   67.4514    ],
       [ 483.94924232,  922.51480481,   67.46432971]])

In [31]:
uncertainty = np.concatenate((
    np.tile(2e-3, (4, 3)),
    np.tile(3e-2, (2, 3))
),
    axis=0,
)
print(uncertainty)

res = np.random.rand(6, 3)
print(res)
sigma0 = 1
weighted = res / (sigma0 * uncertainty) 

print(weighted)

[[0.002 0.002 0.002]
 [0.002 0.002 0.002]
 [0.002 0.002 0.002]
 [0.002 0.002 0.002]
 [0.03  0.03  0.03 ]
 [0.03  0.03  0.03 ]]
[[0.01499121 0.4460505  0.61813125]
 [0.83097974 0.00902343 0.57332214]
 [0.25300696 0.16805087 0.54591013]
 [0.94887571 0.54446653 0.89010659]
 [0.26900976 0.72091693 0.96287684]
 [0.73555439 0.60619201 0.70280101]]
[[  7.49560642 223.0252506  309.06562307]
 [415.48987046   4.51171333 286.66107157]
 [126.50347872  84.02543669 272.95506602]
 [474.43785409 272.23326649 445.05329313]
 [  8.96699206  24.03056427  32.09589476]
 [ 24.51847955  20.20640024  23.4267004 ]]
